# Perbandingan Model AI dengan Data Stunting

Kita akan mencoba melakukan pemodelan dengan XGBoost model

# Inisialisasi Package

In [1]:
# General Package
import io
import logging
import warnings
import pandas as pd
import numpy as np
from copy import deepcopy

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
np.random.seed(1)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
#sns.set_style('darkgrid')
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
dark_style = {
    'figure.facecolor': '#212946',
    'axes.facecolor': '#212946',
    'savefig.facecolor':'#212946',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#2A3459',
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 }
plt.rcParams.update(dark_style)

from pylab import rcParams
rcParams['figure.figsize'] = (12,6)

# Visualization Function

In [3]:
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter, DatetimeTickFormatter
from bokeh.io import curdoc, output_notebook
from bokeh.themes import built_in_themes
from bokeh.plotting import figure, show
from bokeh.palettes import gray
from bokeh.colors import RGB

output_notebook()
curdoc().theme = built_in_themes['caliber']

Loading BokehJS ...

In [13]:
def BokehPlot_1D(data: pd.DataFrame,
                 dates: str = 'date',
                 original: str = 'suhu',
                 Title: str = 'Grafik Suhu Harian',
                 Color: str = '#0a8234',
                 date_unit: str = 'day', # Opsi Argumen: 'year', 'month', 'day'
                ) -> None:
    output_notebook()
    curdoc().theme = built_in_themes['caliber']

    yourdata = data.copy()
    if dates not in yourdata.columns:
        yourdata[dates] = yourdata.index


    # Grafik inisialisasi
    Grafik = figure(title=Title,
                    background_fill_color=RGB(242, 242, 242),
                    background_fill_alpha=0.85,
                    )
    Grafik.width = 1200
    Grafik.height = 400

    # Modifikasi Data
    if date_unit == 'month':
        yourdata['Penanggalan'] = pd.to_datetime(yourdata[dates], format='%Y-%m', errors='coerce')
        Grafik.xaxis.formatter = DatetimeTickFormatter(months="%b %Y")
        x_axis_label = 'Bulan'
        hover_format = "@tanggal{%b %Y}"
    elif date_unit == 'year':
        yourdata['Penanggalan'] = pd.to_datetime(yourdata[dates], format='%Y', errors='coerce')
        Grafik.xaxis.formatter = DatetimeTickFormatter(years="%Y")
        x_axis_label = 'Tahun'
        hover_format = "@tanggal{%Y}"
    else:
        yourdata['Penanggalan'] = pd.to_datetime(yourdata[dates], format='%Y-%m-%d', errors='coerce')
        Grafik.xaxis.formatter = DatetimeTickFormatter(days="%d %b %Y")
        x_axis_label = 'Harian'
        hover_format = "@tanggal{%d %b %Y}"

    Grafik.xaxis.axis_label = x_axis_label
    Grafik.yaxis.axis_label = original

    # Join dataset ke Source
    yourdata.dropna(subset=['Penanggalan'], inplace=True)
    source = ColumnDataSource(data=dict(
        tanggal=yourdata['Penanggalan'],
        Original=yourdata[original],
    ))

    Grafik.line(x='tanggal', y='Original',
                source=source, legend_label=original.upper(),
                line_width=2, color=Color)

    # Hover setting
    Tipo = HoverTool()
    Tipo.tooltips = [(x_axis_label, hover_format),
                     ("Origin",   "@Original{0,.2f}"),
                    ]
    Tipo.formatters = {'@tanggal': 'datetime'}
    Tipo.mode = 'vline'
    Grafik.add_tools(Tipo)

    # Customize plot
    Grafik.title.align = 'center'
    Grafik.title.text_font_size = '20pt'
    Grafik.title.text_color = '#d11950'
    Grafik.legend.location = 'top_left'
    Grafik.legend.click_policy = 'hide'

    Grafik.yaxis.formatter = NumeralTickFormatter(format="0,0")
    show(Grafik)

In [14]:
def Bokeh_Decomposition(DataX : np.ndarray,
                        DataY : np.ndarray,
                        Judul : str = 'Komponen Trend',
                        color : str = "#d11950",
                       ) -> None :
    Data = ColumnDataSource(data=dict(
                              x = DataX,
                              y = DataY,
                            ))

    graphDeretWaktu = figure(width        = 1200,
                             height       = 250,
                             x_axis_type  = "datetime",
                             x_axis_label = "Tanggal",
                             y_axis_label = "Nilai",
                             background_fill_color = gray(20)[17],
                             background_fill_alpha = 0.85,
                            )

    graphDeretWaktu.line('x', 'y',
                         source     = Data,
                         color      = color,
                         line_width = 1,
                        )
    graphDeretWaktu.title.text           = Judul
    graphDeretWaktu.title.text_font_size = "20pt"
    graphDeretWaktu.title.align          = "center"
    graphDeretWaktu.title.text_color     = color
    graphDeretWaktu.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

    THeHover = HoverTool(tooltips=[
                        ("Tanggal", "@x{%F}"),
                        ("Nilai", "@y{0,.2f}")],
                        formatters={"@x": "datetime"
                        })
    graphDeretWaktu.add_tools(THeHover)
    show(graphDeretWaktu)


In [15]:
def BokehPlot(mydata      : pd.DataFrame,
              dates       : str = 'date',
              original    : str = 'Close',
              prediksi    : str = 'Forecasting',
              batas_atas  : str = 'AutoARIMA-hi-95',
              batas_bawah : str = 'AutoARIMA-lo-95',
              modelname   : str = 'ARIMA',
              title       : str = '',
             ) -> None :

    output_notebook()
    curdoc().theme = built_in_themes['caliber']

    yourdata = mydata.copy()
    if dates not in yourdata.columns:
        yourdata[dates] = yourdata.index

    if batas_atas not in yourdata.columns:
        data_dict = {
            'tanggal': yourdata[dates],
            'Original': yourdata[original],
            'Predict': yourdata[prediksi]
        }
        varea = False
    else:
        data_dict = {
            'tanggal': yourdata[dates],
            'Original': yourdata[original],
            'Predict': yourdata[prediksi],
            'upper':yourdata[batas_atas],
            'lower':yourdata[batas_bawah],
        }
        varea = True
    DataSumber = ColumnDataSource(data=data_dict)

    if len(str(title)) <= 5 :
        title = f'Plot {modelname} dalam data Order Demand'
    Grafik = figure(title= title,
                    x_axis_type='datetime',
                    x_axis_label='Tanggal',
                    y_axis_label='Besar Order Demand',
                    background_fill_color=gray(20)[17],
                    background_fill_alpha=0.85,
                    )
    Grafik.width = 1200
    Grafik.height = 400

    Grafik.line(x='tanggal', y='Original',
                source=DataSumber, legend_label='Besar Order Demand pada Gudang',
                line_width=2, color='#252392')
    Grafik.line(x='tanggal', y='Predict',
                source=DataSumber, legend_label=f'{modelname}',
                line_width=2, color='#d40d4c')
    if varea :
        Grafik.varea(x='tanggal',
                     y1='lower',
                     y2='upper',
                     source=DataSumber,
                     color='#61e041',
                     alpha=0.7,
                     legend_label='95% CI'
                    )

    Tipo = HoverTool()
    Tipo.tooltips = [("Tanggal", "@tanggal{%F}"),
                     ("Origin", "@Original{0,.2f}"),
                     (modelname, "@Predict{0,.2f}"),
                     ]
    Tipo.formatters = {'@tanggal': 'datetime'}
    Grafik.add_tools(Tipo)

    Grafik.title.align = 'center'
    Grafik.title.text_font_size = '20pt'
    Grafik.title.text_color = '#d11950'
    Grafik.legend.location = 'top_left'
    Grafik.legend.click_policy = 'hide'
    Grafik.yaxis.formatter = NumeralTickFormatter(format="0,0")

    show(Grafik)

# Initialization from PostGreSQL

In [23]:
from configparser import ConfigParser

confpt = '/mnt/d/WORKING/C01 PROJECTS/Configs.ini'
config = ConfigParser()

config.read(confpt)

['/mnt/d/WORKING/C01 PROJECTS/Configs.ini']

In [47]:
from psycopg2 import connect, DatabaseError

# Database connection parameters
parameter = {"host"    : config.get('Postgre', 'HOST_IP'),
             "database": config.get('Postgre', 'DB_NAME'),
             "user"    : config.get('Postgre', 'DB_USER'),
             "password": config.get('Postgre', 'DB_PWD'),
            }
PostConnected = connect(**parameter)

In [59]:
def GetData(Koneksi   : object,
            query     : str,
            usepandas : bool = False,
           ) -> pd.DataFrame:
    assert len(query) > 10, 'Not sufficient query!'
    if not usepandas :
        postgr = Koneksi.cursor()
        postgr.execute(query)
        raw = postgr.fetchall()
        header = [item[0] for item in postgr.description]
        data = pd.DataFrame(raw, columns = header)
        postgr.close()
    else:
        data = pd.read_sql(query, Koneksi)
    return data

In [65]:
syntax = '''
SELECT
    table_name
FROM
    information_schema.tables
WHERE
    table_schema = 'public'
    AND table_type = 'BASE TABLE';
'''

GetData(PostConnected, syntax)

,table_name
0,countstunting
1,rawdata
2,raw_countstunting
3,xtesting
4,xtraining
5,ytesting
6,ytraining


In [66]:
syntax = 'SELECT * FROM countstunting;'
test = GetData(PostConnected, syntax)
display(test.head(3))

,district_name,stunting,count
0,BENUKI,0,18
1,BENUKI,1,33
2,MAMBERAMO HILIR,0,45


# Training Dataset

In [70]:
X_train = GetData(PostConnected, "SELECT * FROM Xtraining;")
X_test  = GetData(PostConnected, "SELECT * FROM Xtesting;")
y_train = GetData(PostConnected, "SELECT * FROM ytraining;")
y_test  = GetData(PostConnected, "SELECT * FROM ytesting;")

y_train = y_train['stunting'].to_numpy()
y_test  = y_test['stunting'].to_numpy()

In [71]:
print("Ukuran Regressor Training adalah ", X_train.shape)
print("Ukuran Regressor Test adalah ", X_test.shape)
print("Ukuran target Training adalah ", len(y_train))
print("Ukuran target Test adalah ", len(y_test))

Ukuran Regressor Training adalah  (485, 8)
Ukuran Regressor Test adalah  (122, 8)
Ukuran target Training adalah  485
Ukuran target Test adalah  122


In [72]:
X_train.head()

,district_code,gender,malnutrition_level,exclusive_milky,smoke_habit,purewater_access,healthy_toilet,difficult_acess
0,10,0,2,1,0,1,0,1
1,5,1,1,0,0,0,0,0
2,10,0,1,0,0,0,0,1
3,1,1,4,1,1,1,0,1
4,10,1,2,1,0,0,0,1


# XGBoost Modelling

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
model = XGBClassifier(objective        = 'binary:logistic',
                      n_estimators     = 500,
                      max_depth        = 3,
                      learning_rate    = 0.1,
                      subsample        = 0.8,
                      colsample_bytree = 0.8,
                      random_state     = 1,
                      eval_metric      = 'logloss',
                     )

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
OurMetric = accuracy_score(y_test, y_pred)
print(f'Performance akurasi dari Model XGBoost sekarang adalah {OurMetric*100:,.2f}%.')

In [ ]:
from sklearn.metrics import classification_report

def ReportClass_Graph(y_test    : np.ndarray,
                      y_pred    : np.ndarray,
                      forVisual : bool = True,
                     ):
    TheReport = classification_report(y_test, y_pred, output_dict = True)
    TheReport = pd.DataFrame(TheReport).transpose()
    TheReport = TheReport.drop(['accuracy', 'macro avg', 'weighted avg'], errors='ignore')
    Stunts    = {'0': 'Tidak Stunting', '1': 'Stunting'}
    TheReport = TheReport.rename(index = Stunts)
    if not forVisual:
        return TheReport

    plt.figure(figsize=(12, 3))
    ax = TheReport[['precision', 'recall', 'f1-score']].plot(
                kind  = 'bar',
                color = ['#1f77b4', '#ff7f0e', '#2ca02c'],
                width = 0.7,
                alpha = 0.8)
    plt.title('Model XGBoost untuk Prediksi Stunting', pad = 20, fontsize = 18)
    plt.xlabel('Kelas Klasifikasi', labelpad=10)
    plt.ylabel('Score', labelpad=10)
    plt.xticks(rotation=0)
    plt.ylim(0, 1.1)

    for p in ax.patches:
        ax.annotate(
            f'{p.get_height()*100:,.1f}%',
            (p.get_x() + p.get_width() / 2., p.get_height()),
            ha         = 'center',
            va         = 'center',
            xytext     = (0, 10),
            textcoords = 'offset points',
            fontsize   = 10,
            )

    plt.legend(loc = 'upper left')
    plt.tight_layout()

    #plt.savefig('classification_report_stunting.png', dpi=300, bbox_inches='tight')
    plt.show()
    return None

In [ ]:
ReportClass_Graph(y_test, y_pred, True)

In [ ]:
def ConfusionMatrix_Graph(testdata : np.ndarray,
                          prediksi : np.ndarray,
                         ) -> None :
    ConfMax = confusion_matrix(testdata, prediksi)
    case    = ['Tidak Stunting', 'Stunting']

    plt.figure(figsize = (8, 6))
    ax = sns.heatmap(ConfMax,
            annot       = True,
            fmt         = 'd',
            cmap        = 'Blues',
            cbar        = False,
            xticklabels = case,
            yticklabels = case,
            annot_kws   = {'size': 16},
        )

    plt.title('Confusion Matrix Model XGBoost Prediksi', fontsize = 18, pad = 10)
    plt.xlabel('Prediksi', fontsize = 14, labelpad = 15)
    plt.ylabel('Aktual', fontsize = 14, labelpad = 15)

    x, y = ConfMax.shape[0], ConfMax.shape[1]
    ax.axhline(y = 0, color='k', linewidth=2)
    ax.axhline(y = y, color='k', linewidth=2)
    ax.axvline(x = 0, color='k', linewidth=2)
    ax.axvline(x = x, color='k', linewidth=2)

    for i in range(x):
        for j in range(y):
            text_color = 'white' if ConfMax[i, j] > ConfMax.max()/2 else 'black'
            ax.text(j + 0.5, i + 0.5, str(ConfMax[i, j]),
            ha       = 'center',
            va       = 'center',
            color    = text_color,
            fontsize = 14)

    plt.tight_layout()
    plt.show()

In [ ]:
ConfusionMatrix_Graph(y_test, y_pred)

# Optimalkan Parameter XGBoost

In [ ]:
from optuna import create_study as CS
from functools import partial
from sklearn.model_selection import cross_val_score

def SasaranOptimasi(trial : object,
                    X : pd.DataFrame,
                    Y : pd.DataFrame,
                    akurasi : bool = False,
                   ) -> object:
    Parameters = {'objective'        : 'binary:logistic',
                  'eval_metric'      : 'logloss',
                  'n_estimators'     : trial.suggest_int('n_estimators', 1e1, 1e4),
                  'learning_rate'    : trial.suggest_float('learning_rate', 0.01, 0.3),
                  'max_depth'        : trial.suggest_int('max_depth', 3, 10),
                  'subsample'        : trial.suggest_float('subsample', 0.6, 1.0),
                  'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.6, 1.0),
                  'gamma'            : trial.suggest_float('gamma', 0, 0.5),
                  'lambda'           : trial.suggest_float('lambda', 1e-8, 1.0, log = True),
                  'alpha'            : trial.suggest_float('alpha', 1e-8, 1.0, log = True),
                  'seed'             : 1,
                 }
    Models = XGBClassifier(**Parameters)
    if akurasi:
        skor   = cross_val_score(Models, X, Y, cv = 5, scoring = 'accuracy').mean()
    else:
        skor   = cross_val_score(Models, X, Y, cv = 5, scoring = 'f1_weighted').mean()
    return skor

ObjectiveOptim = partial(SasaranOptimasi, X = X_train, Y = y_train)

Optimalkan = CS(direction = 'maximize')
Optimalkan.optimize(ObjectiveOptim, n_trials = 80, show_progress_bar = True)

In [ ]:
print("Pengujian dan Hasilnya")
print("Akurasi sebesar {:.2f}".format(Optimalkan.best_trial.value))
print("Berikut Parameter tersebut:")
for key, value in Optimalkan.best_trial.params.items():
    print("\t{}: {:,.3f}".format(key, value))

In [ ]:
BestParameter = Optimalkan.best_trial.params
BestModel = XGBClassifier(**BestParameter, random_state = 1, enable_categorical = True)
BestModel.fit(X_train, y_train)

In [ ]:
y_predict = BestModel.predict(X_test)
Akurasi = accuracy_score(y_test, y_predict)
ReportClass_Graph(y_test, y_predict, True)

In [ ]:
ConfusionMatrix_Graph(y_test, y_predict)

# Next Steps